In [ ]:
library(ggplot2)
library(igraph)
library(dplyr)
library(RColorBrewer)
library(progress)
library(purrr)
library(cowplot)

library(foreach)
library(doParallel)

source("simpleDynamics.R")

## Single shot dynamics

In [2]:
edge_cmap <- colorRampPalette(c("darkblue", "white"))

In [3]:
# Create the (lattice) graph
L <- 10
d <- 2
g <- make_lattice(length = L, dim = d)

# Hyperparameters and initialization
F <- 10 # Number of features
q <- 4 # Disorder parameter
mode <- "pois"

g <- initialize_features(g, F, q, mode)

In [ ]:
## Dynamics ##
snapshots <- c(1, 1000, 10000, 100000)

# Initialize the progress bar
total_steps <- max(snapshots)
pb <- txtProgressBar(min = 1, max = total_steps, style = 3)

# Loop
for (i in 1:total_steps) {
  if (i %in% snapshots) {
    plotState(g, edge_cmap, i)
  }
  # Perform a single step of the dynamics
  g <- single_step(g)
  
  setTxtProgressBar(pb, i)
}

close(pb) # Close the progress bar

## Analysis

In [ ]:
# Plot dynamics (main report)
ba <- plotDynamics("BA_1", "BA m=1 | F = 10", 10, show=FALSE)
er <- plotDynamics("ER_0-002", "ER p=0.002 | F = 10", 10, show=FALSE)

legend <- get_legend(ba[[1]] + 
  theme(legend.text = element_text(size = 24), legend.title = element_text(size = 24)))

combined_plot <- plot_grid(
  ba[[1]] + theme(legend.position = "none"), 
  ba[[2]] + theme(legend.position = "none"), 
  er[[1]] + theme(legend.position = "none"), 
  er[[2]] + theme(legend.position = "none"), 
  ncol = 2
)

final_plot <- plot_grid(
  legend, combined_plot,
  ncol = 2,
  rel_heights = c(1, 1), rel_widths = c(0.03, 1)
)

ggsave("./temp_data/ba-er.png", final_plot, width = 30, height = 10)

In [ ]:
# Plot dynamics (supplementary material)
labels <- c("resultsLattice", "BA_1", "BA_5", "ER_0-002", "ER_0-007", "ER_0-012")
labels2 <- c("Lattice", "BA m=1", "BA m=5", "ER p=0.002", "ER p=0.007", "ER p=0.012")
F_values <- c(2, 10)

for(i in 1:length(labels)){
  label <- labels[i]
  label2 <- labels2[i]

  
  for (F_value in F_values){
    title <- paste0(label2, " | F = ", F_value)
    p <- plotDynamics(label, title, F_value, show=FALSE)
    
    legend <- get_legend(ba[[1]] + 
      theme(legend.text = element_text(size = 24), legend.title = element_text(size = 24)))

    combined_plot <- plot_grid(
      p[[1]] + theme(legend.position = "none"), 
      p[[2]] + theme(legend.position = "none"), 
      ncol = 2
    )

    final_plot <- plot_grid(
      legend, combined_plot,
      ncol = 2,
      rel_heights = c(1, 1), rel_widths = c(0.03, 1)
    )

    ggsave(paste0("./temp_data/", label, "_", F_value, ".png"), final_plot, width = 30, height = 8)
  }
}

In [ ]:
# Plot phase transition
labels <- c("resultsLattice", "BA_1", "BA_5", "ER_0-002", "ER_0-007", "ER_0-012")
labels2 <- c("Lattice", "BA m=1", "BA m=5", "ER p=0.002", "ER p=0.007", "ER p=0.012")

for (i in 1:length(labels)){
  label <- labels[i]
  label2 <- labels2[i]
  
  plotTransition(label, label2, show=TRUE)
}

In [3]:
# Plot convergence
labels <- c("BA_1", "BA_5", "ER_0-002", "ER_0-007", "ER_0-012")
q_values <- c(1, 10, 100, 200)
F_value <- 10
labels2 <- c("BA m=1", "BA m=5", "ER p=0.002", "ER p=0.007", "ER p=0.012")

final_plot <- plotCulturalConvergence(labels, labels2, q_values, F_value)

ggsave("./temp_data/culturalconvergence.png", plot = final_plot, width = 16, height = 6, units = "in")
